In [1]:
import sys
sys.path.append('../..')
import pandas as pd
import backtrader as bt
from datetime import datetime
import matplotlib.pyplot as plt
from Tools.backtest_tools import *
import plotly.graph_objects as go
import pandas_ta as ta 
from Tools.plotting import CandlePlot

In [2]:
# Load the data and convert the timestamp to datetime
df = pd.read_csv('../../DB/5m/SEIUSDT_5m.csv', parse_dates=True) 

# df['datetime'] = pd.to_datetime(df['datetime'], unit='ms')
# # # Convert 'date' to datetime format
df['datetime'] = pd.to_datetime(df['datetime'])  # Assuming the 'date' is in milliseconds

# # # Set 'date' as index
df.set_index('datetime', inplace=True)

# df.iloc[0]
# df.dtypes
# df.head()
df = df[:].copy()

In [3]:
    
# -- Populate indicators --
df['EMA_slow'] = ta.ema(df.close, length=50)
df['EMA_fast'] = ta.ema(df.close, length=30)
df['rsi'] = ta.rsi(df.close, length=10)
df['ATR']= ta.atr(df.high, df.low, df.close, length=7)
m_bbands = ta.bbands(df.close, length=15, std=1.5) 

df = df.join(m_bbands)        

In [4]:
df

,open,high,low,close,volume,quote_volume,EMA_slow,EMA_fast,rsi,ATR,BBL_15_1.5,BBM_15_1.5,BBU_15_1.5,BBB_15_1.5,BBP_15_1.5
datetime,,,,,,,,,,,,,,,
2023-08-17 06:45:00,0.19067,0.19134,0.19038,0.19038,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-08-17 06:50:00,0.19038,0.19038,0.18868,0.18926,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-08-17 06:55:00,0.18926,0.19136,0.18926,0.19105,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-08-17 07:00:00,0.19105,0.19170,0.18996,0.19042,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-08-17 07:05:00,0.19042,0.19074,0.18971,0.19006,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-09-04 20:45:00,0.27127,0.27187,0.27126,0.27126,0,0,0.271167,0.271210,51.860446,0.000809,0.270211,0.271084,0.271957,0.643859,0.600837
2024-09-04 20:50:00,0.27126,0.27126,0.27067,0.27107,0,0,0.271163,0.271201,49.859846,0.000777,0.270245,0.271107,0.271968,0.635328,0.478712
2024-09-04 20:55:00,0.27107,0.27173,0.27107,0.27124,0,0,0.271166,0.271203,51.711750,0.000761,0.270241,0.271095,0.271949,0.630030,0.585091


In [5]:
def ema_signal(df, current_candle, back_candles):
    df_slice = df.reset_index().copy()
    start = max(0, current_candle - back_candles)
    end = current_candle
    relevant_rows = df_slice.iloc[start:end]

    if all(relevant_rows['EMA_fast'] < relevant_rows['EMA_slow']):
        return 1
    if all(relevant_rows['EMA_fast'] > relevant_rows['EMA_slow']):
        return 2
    return 0

df = df[-30000:-1]
df.reset_index(inplace=True, drop=True)
df['EMA_signal'] = 0
df.loc[20:, 'EMA_signal'] = df.loc[20:].apply(lambda row: ema_signal(df, row.name, 7), axis=1)

/tmp/ipykernel_200915/2826484621.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['EMA_signal'] = 0


In [6]:
df

,open,high,low,close,volume,quote_volume,EMA_slow,EMA_fast,rsi,ATR,BBL_15_1.5,BBM_15_1.5,BBU_15_1.5,BBB_15_1.5,BBP_15_1.5,EMA_signal
0,0.51971,0.52058,0.51895,0.51895,0,0,0.525008,0.521692,41.804231,0.001852,0.518598,0.520353,0.522107,0.674372,0.100279,0
1,0.51895,0.51910,0.51650,0.51790,0,0,0.524729,0.521447,37.760555,0.001959,0.518217,0.520198,0.522179,0.761631,-0.080012,0
2,0.51790,0.51813,0.51642,0.51775,0,0,0.524456,0.521209,37.189555,0.001923,0.517844,0.519947,0.522051,0.809025,-0.022366,0
3,0.51775,0.51882,0.51766,0.51858,0,0,0.524225,0.521039,42.532321,0.001814,0.517753,0.519903,0.522054,0.827274,0.192321,0
4,0.51858,0.51862,0.51789,0.51836,0,0,0.523995,0.520866,41.492856,0.001659,0.517678,0.519871,0.522065,0.843743,0.155448,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29994,0.27076,0.27127,0.27069,0.27127,0,0,0.271163,0.271206,51.959202,0.000842,0.270150,0.271039,0.271928,0.656043,0.630099,2
29995,0.27127,0.27187,0.27126,0.27126,0,0,0.271167,0.271210,51.860446,0.000809,0.270211,0.271084,0.271957,0.643859,0.600837,2
29996,0.27126,0.27126,0.27067,0.27107,0,0,0.271163,0.271201,49.859846,0.000777,0.270245,0.271107,0.271968,0.635328,0.478712,2
29997,0.27107,0.27173,0.27107,0.27124,0,0,0.271166,0.271203,51.711750,0.000761,0.270241,0.271095,0.271949,0.630030,0.585091,2


In [7]:
def total_signal(df, current_candle, backcandles):
    if (ema_signal(df, current_candle, backcandles)==2
        and df.close[current_candle]<=df['BBL_15_1.5'][current_candle]
        #and df.RSI[current_candle]<60
        ):
            return 2
    if (ema_signal(df, current_candle, backcandles)==1
        and df.close[current_candle]>=df['BBU_15_1.5'][current_candle]
        #and df.RSI[current_candle]>40
        ):
    
            return 1
    return 0
        
df['TotalSignal'] = df.apply(lambda row: total_signal(df, row.name, 7), axis=1)


/tmp/ipykernel_200915/1293293665.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['TotalSignal'] = df.apply(lambda row: total_signal(df, row.name, 7), axis=1)


In [8]:
def pointpos(x):
    if x['TotalSignal']==2:
        return x['low']-1e-3
    elif x['TotalSignal']==1:
        return x['high']+1e-3
    else:
        return np.nan

df['pointpos'] = df.apply(lambda row: pointpos(row), axis=1)

/tmp/ipykernel_200915/514633494.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['pointpos'] = df.apply(lambda row: pointpos(row), axis=1)


In [11]:
dfplt = df[450:700]
cp = CandlePlot(dfplt)
cp.add_points()
cp.add_traces(['EMA_slow', 'EMA_fast', 'BBL_15_1.5', 'BBU_15_1.5', 'BBM_15_1.5'])


In [12]:
cp.show_plot()